# Resource Validation

In [29]:
LOCAL="http://localhost:8088/fhir" # Jupyter uses 8888!!
JHU="http://20.119.216.32:8000/r4"
server_path=LOCAL

import httpx
import json
from IPython.display import display, HTML

client = httpx.Client()
request_headers = {
    "Content-Type" : "application/fhir+json; charset=UTF-8",
    "Accept-Charset" : "utf-8",
    "User-Agent" : "HAPI-FHIR/5.0.0 (FHIR Client; FHIR 4.0.1/R4; apache)" }

def do_hapi_request(internal_path, in_params):
    rq = httpx.Request("GET", server_path + internal_path,
               params = in_params,  headers=request_headers)
    print(rq.url)
    response = client.send(rq)
    return response

def do_hapi_post(internal_path, form_data, in_params):
    rq = httpx.Request("POST", server_path + internal_path,
                       data=form_data,
                       params=in_params,  
                       headers=request_headers)
    print(rq.url)
    response = client.send(rq)
    return response
    
def decode_status(response):
    if response is None:
        print("NO RESPONSE")
        return(True)
    else:
        if (response.status_code >= 200 and response.status_code < 300):
            return(False)
        elif (response.status_code >= 400 and response.status_code < 500):
            return(True)
        elif (response.status_code >= 500) :
            return(True)

def decode_code_system_json(response):
    if not decode_status(response) :
        response_dict = json.loads(response.text)
        if ('entry' in response_dict):
            print("title:" + response_dict['entry'][0]['resource']['title'])
            print("resourceType:", response_dict['entry'][0]['resource']['resourceType'],
                 end="  ")
            print("id: " + response_dict['entry'][0]['resource']['id'])
            print("url: " + response_dict['entry'][0]['resource']['url'])
            print("full url: " + response_dict['entry'][0]['fullUrl'])
            if len(response_dict['entry']) > 1 :
                print("identifier system:\"" + response_dict['entry'][1]['resource']['identifier'][0]['system'],
                      end="\",  ")
                print("value: \"" + response_dict['entry'][1]['resource']['identifier'][0]['value'] + "\"")
            else:
                print("?? no 2nd entry for resource identifier")
        else:
            print("Not Present")
        print("")  
        
def decode_organization_json(response):
    if not decode_status(response) :
        response_dict = json.loads(response.text)
        if ('entry' in response_dict):
            print("name:" + response_dict['entry'][0]['resource']['name'])
            print("resourceType:", response_dict['entry'][0]['resource']['resourceType'],
                 end="  ")
            print("id: " + response_dict['entry'][0]['resource']['id'])
        else:
            print("Not Present")
        print("") 
        
def decode_patient_json(response):
    if not decode_status(response) :
        response_dict = json.loads(response.text)
        if ('entry' in response_dict):
            print("name:" + response_dict['entry'][0]['resource']['name'][0]['family'])
            print("name:" + response_dict['entry'][0]['resource']['name'][0]['given'][0])            
            print("resourceType:", response_dict['entry'][0]['resource']['resourceType'],
                 end="  ")
            print("id: " + response_dict['entry'][0]['resource']['id'])
        else:
            print("Not Present")
        print("")
        
def decode_encounter_json(response):
    if not decode_status(response) :
        response_dict = json.loads(response.text)
        if ('entry' in response_dict):
            print(response_dict['entry'][0]['fullUrl'])   
            print("id: " + response_dict['entry'][0]['resource']['id'])
            print("text status: " + response_dict['entry'][0]['resource']['text']['status'])
            print("text div: " + response_dict['entry'][0]['resource']['text']['div'])            
            print("status: " + response_dict['entry'][0]['resource']['status'])
        else:
            print("Not Present")
        print("")        

def decode_validate_response(repsonse) :
    print(response)
    if False:
        print(response.keys())
        print(response['issue']['severity'])
        print(response['issue']['response'])
        display(HTML(response['text']['div']))
        print("")
        
def decode_lookup_response(response):
    if (not decode_status(response)) :
        response_dict = json.loads(response.text)
        for p in response_dict['parameter']:
            if p['name'] != 'property' and p['name'] != 'designation':
                if 'valueString' in p:
                    print(p['name'], ":", p['valueString'], sep="", end=", ")
                if 'valueBoolean' in p:
                    print(p['name'], ":", p['valueBoolean'], sep="", end=", ")
        print("")
    else:
        print("error")
    print("")

## Organization  

In [2]:
org_file = open("0_organization.json", "r")
form_data = org_file.read()

response = do_hapi_post("/Organization/", form_data, 
             {"_format" : "json", "_pretty" : "true"})    
if (not decode_status(response)) :
    decode_organization_json(response)  
else:
    print("no good")
print(response)    

http://localhost:8088/fhir/Organization/?_format=json&_pretty=true
Not Present

<Response [201 Created]>


In [3]:
response = do_hapi_request("/Organization/", {'name' : 'HealthGorilla'})
if (not decode_status(response)) :
    decode_organization_json(response)  
else:
    print("no good")

http://localhost:8088/fhir/Organization/?name=HealthGorilla
name:HealthGorilla
resourceType: Organization  id: 18829



## Patient

In [4]:
patient_file = open("1_simple_patient.json", "r")
form_data = patient_file.read()
response = do_hapi_post("/Patient/", form_data, 
             {"_format":"json", "_pretty":"true"})    
if (not decode_status(response)) :
    decode_code_system_json(response) 
else:
    print("no good")
print(response)

http://localhost:8088/fhir/Patient/?_format=json&_pretty=true
Not Present

<Response [201 Created]>


In [5]:
response = do_hapi_request("/Patient/", {'name' : 'Chalmers'})
if (not decode_status(response)) :
    decode_patient_json(response)  
else:
    print("no good")

http://localhost:8088/fhir/Patient/?name=Chalmers
[{'fullUrl': 'http://localhost:8088/fhir/Patient/18845', 'resource': {'resourceType': 'Patient', 'id': '18845', 'meta': {'versionId': '1', 'lastUpdated': '2022-10-04T17:30:41.144-06:00', 'source': '#5gAIj595LeemfHvm'}, 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Peter James <b>CHALMERS </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>25 December 1974</span></td></tr></tbody></table></div>'}, 'active': True, 'name': [{'use': 'official', 'family': 'Chalmers', 'given': ['Peter', 'James']}], 'telecom': [{'system': 'phone', 'value': '(03) 5555 6473', 'use': 'work', 'rank': 1}, {'system': 'phone', 'value': '(03) 3410 5613', 'use': 'mobile', 'rank': 2}], 'gender': 'male', 'birthDate': '1974-12-25', 'deceasedBoolean': False}, 'search': {'mode': 'match'}}, {'fullUrl': 'http://localhost:8088/fhir/Patient/18846', 'resource': {'resourceType': 'Pat

## Encounter

In [6]:
encounter_file = open("2_encounter.json", "r")
form_data = encounter_file.read()
response = do_hapi_post("/Encounter/", form_data, 
             {"_format":"json", "_pretty":"true"})    
if (not decode_status(response)) :
    decode_code_system_json(response) 
else:
    print("no good")
print(response)

http://localhost:8088/fhir/Encounter/?_format=json&_pretty=true
Not Present

<Response [201 Created]>


In [30]:
response = do_hapi_request("/Encounter/", {'patient' : '18845'})
if (not decode_status(response)) :
    decode_encounter_json(response)  
else:
    print("no good")

http://localhost:8088/fhir/Encounter/?patient=18845
http://localhost:8088/fhir/Encounter/18852
id: 18852
text status: generated
text div: <div xmlns="http://www.w3.org/1999/xhtml">Encounter with patient</div>
status: in-progress

